In [ ]:


!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1

!pip install -q diffusers transformers accelerate
!pip install -q ultralytics==8.1.0
!pip install -q autodistill autodistill-grounding-dino roboflow

print("Libraries installed. PLEASE RESTART RUNTIME!")

In [ ]:
# FIX: Downgrade PyTorch to a YOLOv8-compatible version

!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1

In [ ]:
# ==============================
# Block 1: Mount Google Drive & create project folders
# ==============================

from google.colab import drive
import os

drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/ChildSafetyProject_SDXL"

RAW_IMAGES_DIR = os.path.join(PROJECT_ROOT, "raw_images")
YOLO_DATASET_DIR = os.path.join(PROJECT_ROOT, "yolo_dataset")

os.makedirs(RAW_IMAGES_DIR, exist_ok=True)
os.makedirs(YOLO_DATASET_DIR, exist_ok=True)

print("Project folders created:")
print(PROJECT_ROOT)


In [ ]:
# ==============================
# Block 3: SDXL image generation (children rooms)
# ==============================

import torch
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler
import os
import random

MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = StableDiffusionXLPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

pipe.enable_model_cpu_offload()
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

NEGATIVE_PROMPT = (
    "cartoon, illustration, drawing, anime, low quality, blurry, "
    "distorted, unrealistic, deformed, extra objects"
)

prompts = {
    "scissors": [
        "A realistic photo of a child's bedroom with toys and a bed, "
        "sharp metal scissors lying on the floor near colorful toys, "
        "natural daylight, photorealistic"
    ],
    "battery": [
        "A realistic nursery room with toys, "
        "a small button battery on the carpet near children's toys, "
        "natural lighting, photorealistic"
    ],
    "detergent": [
        "A realistic children's playroom, "
        "a colorful detergent bottle on a low shelf within child's reach, "
        "natural light, photorealistic"
    ],
    "wires": [
        "A realistic child's bedroom with toys, "
        "a damaged electrical power cord with exposed wires on the floor, "
        "natural lighting, realistic photo"
    ]
}

IMAGES_PER_CLASS = 25

for label, plist in prompts.items():
    for i in range(IMAGES_PER_CLASS):
        image = pipe(
            prompt=random.choice(plist),
            negative_prompt=NEGATIVE_PROMPT,
            num_inference_steps=30,
            guidance_scale=7.0,
            height=1024,
            width=1024
        ).images[0]

        image.save(os.path.join(RAW_IMAGES_DIR, f"{label}_{i}.jpg"))

print("SDXL image generation completed.")


In [ ]:
# ==============================
# Block 4: Automatic labeling
# ==============================

from autodistill_grounding_dino import GroundingDINO
from autodistill.detection import CaptionOntology
import os

# Ensure PROJECT_ROOT, RAW_IMAGES_DIR, YOLO_DATASET_DIR are defined for standalone execution
# This assumes Google Drive is already mounted (from a previous cell or manually).
PROJECT_ROOT = "/content/drive/MyDrive/ChildSafetyProject_SDXL"
RAW_IMAGES_DIR = os.path.join(PROJECT_ROOT, "raw_images")
YOLO_DATASET_DIR = os.path.join(PROJECT_ROOT, "yolo_dataset")
os.makedirs(RAW_IMAGES_DIR, exist_ok=True)
os.makedirs(YOLO_DATASET_DIR, exist_ok=True)

ontology = CaptionOntology({
    "scissors": "Sharp Object",
    "battery": "Choking Hazard",
    "detergent bottle": "Chemical Danger",
    "exposed electrical wire": "Electrical Hazard"
})

model = GroundingDINO(ontology=ontology)

model.label(
    input_folder=RAW_IMAGES_DIR,
    output_folder=YOLO_DATASET_DIR,
    extension=".jpg"
)

print("Labeling completed.")

# --- Added check ---
# Verify if images were actually written to the YOLO_DATASET_DIR/images directory
output_images_path = os.path.join(YOLO_DATASET_DIR, "images")
if os.path.exists(output_images_path):
    labeled_images = [f for f in os.listdir(output_images_path) if f.endswith(".jpg")]
    print(f"Found {len(labeled_images)} labeled images in {output_images_path} after labeling.")
    if not labeled_images:
        print("WARNING: The labeled images directory is empty. This might cause issues in subsequent steps.")
else:
    print(f"WARNING: The output images directory {output_images_path} was not created.")


In [ ]:
 #==============================
# Block 5: Train/Val split
# ==============================

import os, shutil, random
random.seed(42)

images_dir = os.path.join(YOLO_DATASET_DIR, "images")
labels_dir = os.path.join(YOLO_DATASET_DIR, "labels")

for s in ["train", "val"]:
    os.makedirs(os.path.join(images_dir, s), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, s), exist_ok=True)

imgs = [f for f in os.listdir(images_dir) if f.endswith(".jpg")]
random.shuffle(imgs)

split = int(0.8 * len(imgs))
train, val = imgs[:split], imgs[split:]

def move(imgs, split):
    for img in imgs:
        shutil.move(os.path.join(images_dir, img), os.path.join(images_dir, split, img))
        shutil.move(
            os.path.join(labels_dir, img.replace(".jpg", ".txt")),
            os.path.join(labels_dir, split, img.replace(".jpg", ".txt"))
        )

move(train, "train")
move(val, "val")

print("Train/Val split done.")


In [ ]:
# ==============================
# Block 6: Create data.yaml
# ==============================

import yaml
import os

yaml_path = os.path.join(YOLO_DATASET_DIR, "data.yaml")

data = {
    "path": YOLO_DATASET_DIR,
    "train": "train/images",  # Corrected path based on Autodistill's output structure
    "val": "valid/images",    # Corrected path based on Autodistill's output structure
    "names": [
        "Sharp Object",
        "Choking Hazard",
        "Chemical Danger",
        "Electrical Hazard"
    ]
}

with open(yaml_path, "w") as f:
    yaml.dump(data, f)

print("data.yaml created.")


In [ ]:
# ==============================
# Block 7: Train YOLOv8
# ==============================

from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=8,
    device=0,
    project="child_safety_project_sdxl", # Changed from PROJECT_ROOT to a valid project name
    name="yolov8_child_safety_sdxl"
)

print("Training finished.")


In [ ]:
# ==============================
# Block 8: Evaluation
# ==============================

metrics = model.val(data=yaml_path)

print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.p.mean():.4f}")
print(f"Recall: {metrics.box.r.mean():.4f}")


In [ ]:
# ==============================
# Block 9: Inference visualization
# ==============================

import os
import matplotlib.pyplot as plt # Import matplotlib for displaying images

test_imgs = [os.path.join(RAW_IMAGES_DIR, f) for f in os.listdir(RAW_IMAGES_DIR)[:5]]

for img_path in test_imgs:
    results = model(img_path, conf=0.3)
    # Use .plot() to get the annotated image as a NumPy array
    # Then display it using matplotlib
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        plt.imshow(im_array[..., ::-1])  # Convert BGR to RGB for matplotlib
        plt.axis('off') # Hide axes
        plt.show() # Display the image


In [ ]:
# ==============================
# BLOCK 9: Inference & visualization
# ==============================

from IPython.display import Image, display # Import Image from IPython.display
from PIL import Image as PILImage # Alias PIL.Image to avoid conflict with IPython.display.Image

test_images = [os.path.join(RAW_IMAGES_DIR, f) for f in os.listdir(RAW_IMAGES_DIR) if f.endswith(".jpg")][:5]

for img_path in test_images:
    print(f"Processing: {img_path}")
    results = model(img_path, conf=0.3)
    # Get the annotated image (numpy array)
    annotated_image_array = results[0].plot()
    # Convert numpy array to PIL Image
    annotated_image_pil = PILImage.fromarray(annotated_image_array)
    # Display the PIL Image
    display(annotated_image_pil)